# Micro Projects

## Bulk File Renaming

### Micro Project #1: General Purpose Renaming Function

#### Define the renaming function

The function takes three arguments: the prefix the new filenames should have, the extension of the files to be renamed, and the directory in which the files are located.

In [ ]:
import os
def rename_files(prefix, ext, path):
    i =0 
    file_list = os.listdir(path)
    for filename in file_list:
        new = prefix + str(i) + ext
        source = path + filename
        new = path + new
        os.rename(source, new)
        i += 1


#### Using the function above

In [ ]:
rename_files("image", ".png", os.getcwd())

### Micro Project #2: XML-specific Function

#### Define the XML-specific function for individualized names 
I created this one for a very specific use-case where I had a bunch of XML files that I had to analyze and one of the first cleaning steps wanted to do was rename them so that they actually had helpful and informative names as this would help me in later data cleaning steps.

In [ ]:
import os
import xml.etree.ElementTree as ET

pathname = os.getcwd() # Directory of files to be renamed
# If wd is not the same as 'pathname' then change the pathname above to the correct directory

last_name = ''
location = ''
i = 0

file_list = os.listdir(pathname)

for filename in file_list:
    tree = ET.parse(pathname + filename)
    root = tree.getroot()
    for elem in root.iter():
        if 'PersonSurName' in elem.tag:
            last_name = elem.text
        if 'LocationState' in elem.tag:
            location = elem.text
    new = str(last_name) + "-" + str(location) + ".xml"
    source = pathname + filename
    new = pathname + new
    try:
        os.rename(source, new)
    except FileExistsError:
        new = str(last_name) + "-" + str(location) + str(i) + ".xml"
        os.rename(source, new)
        
    i += 1

## Web Scraping


### Micro Project #3: Automated Google Scholar Search

In [16]:
import requests, json
import pandas as pd
from plyer import notification
from secrets import keys


searchTerms = 'sadism psychopathy'
key = keys["serpApiKEY"]
yr_low = ''
yr_high = ''
last_year = '1' # Options: 0 = sorted by relevance, 1 = abstracts only, 2 = everything
num = '20'
lang = 'lang_en'

url = 'https://serpapi.com/search?engine=google_scholar'

params = {'engine' : 'google_scholar',
          'q' : searchTerms,
          'api_key': key,
          'as_ylo' : yr_low,
          'as_yhi' : yr_high,
          'scisbd' : last_year,
          'num' : num,
          'lr' : lang}

response = requests.request("GET", url, params = params)

scholar_results =json.loads(response.text)


In [73]:
scholar_results.keys()

dict_keys(['search_metadata', 'search_parameters', 'search_information', 'organic_results', 'pagination', 'serpapi_pagination'])

In [ ]:
titles = []
links = []
summaries = []

for i in scholar_results['organic_results']:
    titles.append(i['title'])
    links.append(i['link'])
    summaries.append(i['snippet'])
    for i in summaries:
        i.strip('\W')

df = pd.DataFrame({'Title': titles, 'Link': links, 'Summary' : summaries})
df

### Micro Project #4: Weather Updates

In [ ]:
import pandas as pd
import requests, json, datetime
from plyer import notification
from secrets import keys

lat=38.3357
lon=-77.4342
part="minutely"
APIkey= keys["openWeatherKEY"]

url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={APIkey}'

querystring = {"units":"imperial"}

response = requests.request("GET", url, params = querystring)

weather_dict = json.loads(response.text)

def get_alerts():
    if 'alerts' not in weather_dict.keys():
        alerts = "No weather alerts"
    else:
        alerts = []
        for i in weather_dict['alerts']:
            alerts.append(i['event'])
    for i in alerts:
        return i
    
sunset = datetime.datetime.fromtimestamp(weather_dict['current']['sunset'])
sunset = sunset.time()
alert_start = datetime.datetime.fromtimestamp(weather_dict['alerts'][0]['start'])
alert_end = datetime.datetime.fromtimestamp(weather_dict['alerts'][0]['end'])
desc = weather_dict['current']['weather'][0]['description']
temp = weather_dict['current']['temp']
alerts = get_alerts()


In [6]:
weather_dict.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'hourly', 'daily'])

#### Create a notification for weather updates

In [34]:
notification.notify(title = "Weather Update", 
                    message = f"It is currently {temp} deg F and {desc}\nThe sun will set at approximately {sunset}\n{alerts} from {alert_start} until {alert_end}", 
                    timeout=15)